# Baseline Models

We will develop the baseline neural network model (one hidden layer, 3-node softmax output) for comparisons for later architectures.

In [1]:
# Imports
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import scale
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


In [2]:
# Reading in Data
data_path = 'data/upsampled_data.csv'
data_final = pd.read_csv(data_path)
data_final = data_final.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1)

In [3]:
data_final.columns

Index(['protein', 'resnum', 'aa1_pi', 'aa2_pi', 'delta_pi', 'grantham',
       'aa1_weight', 'aa2_weight', 'delta_weight', 'aa1_volume',
       ...
       'secondary_structure1_is_bend_6', 'secondary_structure1_is_coil_6',
       'secondary_structure1_is_het_6', 'secondary_structure1_is_unknown_6',
       'secondary_structure2_is_helix_6', 'secondary_structure2_is_beta_6',
       'secondary_structure2_is_coil_6', 'secondary_structure2_is_het_6',
       'secondary_structure2_is_unknown_6', 'type'],
      dtype='object', length=971)

In [10]:
# LOPO Splitting
def train_test_split(protein, curr_data):
    train_data = curr_data[curr_data.protein != protein].drop(['protein', 'pdb', 'resnum'], axis=1)
    test_data = curr_data[curr_data.protein == protein].drop(['protein', 'pdb', 'resnum'], axis=1)
    
    y_train = train_data.type
    encoder = LabelEncoder()
    encoder.fit(y_train)
    encoded_y_train = encoder.transform(y_train)
    y_train_oh = np_utils.to_categorical(encoded_y_train)
    
    scaler_train = StandardScaler()
    x_train = train_data.drop(['type'], axis=1)
    x_columns = x_train.columns
    x_train = scaler_train.fit_transform(x_train)
    x_train = pd.DataFrame(x_train, columns=x_columns)
    
    y_test = test_data.type
    encoder = LabelEncoder()
    encoder.fit(y_test)
    encoded_y_test = encoder.transform(y_test)
    y_test_oh = np_utils.to_categorical(encoded_y_test)
    
    scaler_test = StandardScaler()
    x_test = test_data.drop(['type'], axis=1)
    x_test = scaler_test.fit_transform(x_test)
    x_test = pd.DataFrame(x_test, columns=x_columns)

    return x_train, y_train_oh, x_test, y_test_oh

In [5]:
# Baseline Model
def baseline_model():
    model = Sequential()
    model.add(Dense(256, input_dim = 507, activation = 'relu'))
    model.add(Dense(3, activation='softmax'))
    opt = optimizers.Adam(learning_rate = 0.01)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    return(model)

In [11]:
# Baseline Analysis
def baseline(protein):
    # Split Training Data based on Protein
    x_train, y_train, x_test, y_test = train_test_split(protein, data_final)
    
    # Build Model
    # 40 epochs based on initial runs and graphing of loss over time.
    estimator = KerasClassifier(build_fn=baseline_model, epochs=20, batch_size=64, verbose=1)
    estimator.fit(x_train, y_train, verbose=1)
    
    # Generate Analysis Characteristics
    probs = estimator.predict_proba(x_test)
    predictions = estimator.predict(x_test)
    precision = precision_score(np.argmax(y_test,1), predictions, average="macro", zero_division=0)
    score = estimator.score(x_test, y_test)

    print('Accuracy: {}'.format(score))
    print('Precision: {}'.format(precision))
    
    # Plotting Confusion Matrices
    cmatrix = confusion_matrix(np.argmax(y_test,1), predictions)
    fig, ax = plt.subplots()
    sns.heatmap(cmatrix, xticklabels = ['Beneficial', 'Deleterious', 'Neutral'], yticklabels = ['Beneficial', 'Deleterious', 'Neutral'])
    ax.set_title('{}'.format(protein))
    plt.savefig('img/baseline-upsampled-cmatrix/' + protein + '-baseline-upsampled-cmatrix.png', dpi=300)

In [12]:
proteins = list(data_final['protein'].unique())
for protein in proteins:
    baseline(protein)

Epoch 1/5
44187/44187 [==============================] - 6s 134us/step - loss: 162.3191 - accuracy: 0.3308
Epoch 2/5
44187/44187 [==============================] - 6s 132us/step - loss: 1.1034 - accuracy: 0.3377
Epoch 3/5
44187/44187 [==============================] - 6s 134us/step - loss: 1.1044 - accuracy: 0.3340
Epoch 4/5
44187/44187 [==============================] - 6s 131us/step - loss: 1.1041 - accuracy: 0.3331
Epoch 5/5
33531/33531 [==============================] - 1s 31us/step


NameError: name 'np' is not defined